In [ ]:
!pip install pykeen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
from torch.optim import Adam
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
from pykeen.nn.representation import Representation
from pykeen.models import ComplEx, RotatE, TransE
from pykeen.evaluation import RankBasedEvaluator
from pykeen.sampling import BasicNegativeSampler
from pykeen.losses import Loss

INFO:pykeen.utils:Using opt_einsum


In [ ]:
class CRRLoss(Loss):
    def __init__(self, t=1.0, p=0.1):
        super().__init__()
        self.t = t  # Temperature (tau)
        self.p = p  # Pressure (rho)

    def cliff_sigmoid(self, x):
        return 1 / (1 + torch.exp((self.p - x) / self.t))

    def forward(self, scores, labels):
        # scores: (batch_size, num_negatives + 1) - includes positive and negative scores
        # labels: (batch_size, num_negatives + 1) - 1 for positive, 0 for negative
        pos_scores = scores[labels == 1]  # Extract positive scores
        neg_scores = scores[labels == 0]  # Extract negative scores
        # Reshape for broadcasting: (batch_size, 1) - (batch_size, num_negatives)
        diff = neg_scores - pos_scores.unsqueeze(-1)
        return torch.log(torch.sum(self.cliff_sigmoid(diff), dim=1) + 1.0).mean()

# all-MiniLM-L6-v2 embedding

In [ ]:
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = model.to(device)

NameError: name 'model' is not defined

In [ ]:
df = pd.read_csv("/content/triples.tsv", sep="\t", header=None, names=["head", "relation", "tail"])
entities = set(df["head"]).union(set(df["tail"]))

In [ ]:
entity_to_embedding = {}

In [ ]:
with torch.no_grad():
    for entity in tqdm(entities):
        inputs = tokenizer(entity, return_tensors="pt", truncation=True, padding=True).to(device)
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
        entity_to_embedding[entity] = embedding.squeeze(0)

100%|██████████| 82586/82586 [06:38<00:00, 207.25it/s]


In [ ]:
tf = TriplesFactory.from_path("triples.tsv", delimiter="\t")

In [ ]:
class PrecomputedEmbedding(Representation):
    def __init__(self, entity_tensor: torch.FloatTensor, trainable: bool = False):
        num_entities, embedding_dim = entity_tensor.shape
        super().__init__(max_id=num_entities, shape=(embedding_dim,))
        self.embeddings = torch.nn.Parameter(entity_tensor, requires_grad=trainable)

    def _plain_forward(self, indices: torch.LongTensor) -> torch.FloatTensor:
        return self.embeddings[indices]

In [ ]:
entity_tensor = torch.stack([entity_to_embedding[entity] for entity in tf.entity_to_id])
embedder = PrecomputedEmbedding(entity_tensor, trainable=True)

INFO:pykeen.nn.representation:Inferred unique=True for PrecomputedEmbedding()


In [ ]:
train_factory, valid_factory, test_factory = tf.split([0.8, 0.1, 0.1])

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [380469, 57314, 57315]


In [ ]:
model = RotatE(triples_factory=tf, embedding_dim=384)

INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()


In [ ]:
optimizer = Adam(params=model.get_grad_params(), lr=0.0001)

In [ ]:
result = pipeline(
    training=train_factory,
    validation=valid_factory,
    testing=test_factory,
    model=model,
    model_kwargs=dict(
        entity_representations=embedder,
    ),
    evaluator=RankBasedEvaluator,
    loss=CRRLoss(),
    training_kwargs=dict(
        num_epochs=100,
        batch_size=1024,
    ),
    evaluator_kwargs=dict(
        batch_size=1024,
    ),
)

print(f"Hits@1: {result.metric_results.get_metric('hits_at_1'):.4f}")
print(f"Hits@3: {result.metric_results.get_metric('hits_at_3'):.4f}")
print(f"Hits@10: {result.metric_results.get_metric('hits_at_10'):.4f}")
print(f"MRR: {result.metric_results.get_metric('mrr'):.4f}")
print(f"MR: {result.metric_results.get_metric('mr'):.4f}")

INFO:pykeen.pipeline.api:Using device: None


Training epochs on cuda:0:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/448 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/448 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/448 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/448 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/448 [00:00<?, ?batch/s]

Evaluating on cuda:0:   0%|          | 0.00/57.3k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 372.21s seconds


Hits@1: 0.0950
Hits@3: 0.1554
Hits@10: 0.2201
MRR: 0.1409
MR: 2847.9385


# BERT Embeddings

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
entity_to_embedding = {}

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
'dosher <__> NOUN'.split(" ")[0].strip()

'dosher'

In [ ]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
model = model.to(device)

In [ ]:
with torch.no_grad():
    for entity in tqdm(entities):
        word_only = str(entity).split(" ")[0].strip()
        inputs = tokenizer(word_only, return_tensors="pt", truncation=True, padding=True).to(device)
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
        entity_to_embedding[str(entity)] = embedding.squeeze(0)

100%|██████████| 82586/82586 [12:01<00:00, 114.45it/s]


In [ ]:
import json

In [ ]:
entity_to_embedding_serializable = {
    k: v.tolist() for k, v in entity_to_embedding.items()
}

In [ ]:
with open('politeness_bert_embeddings.json', 'w') as f:
    json.dump(entity_to_embedding_serializable, f, indent=4)

In [ ]:
entity_tensor = torch.stack([entity_to_embedding[str(entity)] for entity in tf.entity_to_id])
embedder = PrecomputedEmbedding(entity_tensor, trainable=True)

INFO:pykeen.nn.representation:Inferred unique=True for PrecomputedEmbedding()


In [ ]:
embedder = PrecomputedEmbedding(entity_tensor, trainable=False)

INFO:pykeen.nn.representation:Inferred unique=True for PrecomputedEmbedding()


In [ ]:
train_factory, valid_factory, test_factory = tf.split([0.8, 0.1, 0.1])

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [380469, 57314, 57315]


In [ ]:
model = RotatE(triples_factory=tf, embedding_dim=768)

INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()


In [ ]:
optimizer = Adam(params=model.get_grad_params(), lr=0.0001)

In [ ]:
result = pipeline(
    training=train_factory,
    validation=valid_factory,
    testing=test_factory,
    model=model,
    model_kwargs=dict(
        entity_representations=embedder,
    ),
    evaluator=RankBasedEvaluator,
    loss=CRRLoss(),
    training_kwargs=dict(
        num_epochs=100,
        batch_size=1024,
    ),
    evaluator_kwargs=dict(
        batch_size=1024,
    ),
)

print(f"Hits@1: {result.metric_results.get_metric('hits_at_1'):.4f}")
print(f"Hits@3: {result.metric_results.get_metric('hits_at_3'):.4f}")
print(f"Hits@10: {result.metric_results.get_metric('hits_at_10'):.4f}")
print(f"MRR: {result.metric_results.get_metric('mrr'):.4f}")
print(f"MR: {result.metric_results.get_metric('mr'):.4f}")

INFO:pykeen.pipeline.api:Using device: None


Training epochs on cuda:0:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/448 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/448 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/448 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/448 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/448 [00:00<?, ?batch/s]

Evaluating on cuda:0:   0%|          | 0.00/57.3k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 723.59s seconds


Hits@1: 0.0928
Hits@3: 0.1537
Hits@10: 0.2183
MRR: 0.1387
MR: 3401.4739


# PyKEEN TextRepresentation

In [ ]:
from pykeen.models import ERModel
from pykeen.nn import TextRepresentation

In [ ]:
tf = TriplesFactory.from_path("triples.tsv", delimiter="\t")

In [ ]:
train_factory, valid_factory, test_factory = tf.split([0.8, 0.1, 0.1])

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [380469, 57314, 57315]


In [ ]:
entity_representations = TextRepresentation.from_triples_factory(
    triples_factory=tf,
    encoder="transformer",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

0it [00:00, ?it/s]

INFO:pykeen.nn.representation:Inferred unique=True for TextRepresentation()


In [ ]:
result = pipeline(
    training=train_factory,
    validation=valid_factory,
    testing=test_factory,
    model=ERModel,
    model_kwargs=dict(
        interaction="ermlpe",
        interaction_kwargs=dict(
            embedding_dim=entity_representations.shape[0],
        ),
        entity_representations=entity_representations,
        relation_representations_kwargs=dict(
            shape=entity_representations.shape,
        ),
    ),
    training_kwargs=dict(num_epochs=1, batch_size=1024),
    evaluator_kwargs=dict(
        batch_size=1024,
    ),
)

INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.training.training_loop:Starting sub_batch_size search for training now...
INFO:pykeen.training.training_loop:This model does not support sub-batching.


MemoryError: The current model can't be trained on this hardware with these parameters.